In [19]:
from arrange_blocks import Reader
import compile_defs
import re
import spacy
from QANet import eval
nlp = spacy.load("en_core_web_sm")

In [12]:
defs = compile_defs.run("./definitions")
qa_config_path = "config.json"
qa_extractor = eval.Eval(qa_config_path)

In [13]:
tmp_dir = "tmp"
files = os.listdir(tmp_dir)

Number of Pages:  1
['Page Header\nRandom Text Here\n123, Abc St, Place,\nState, Country\n85281\nPREPARED FOR\nGenesis AI Pvt. Ltd.\nAhmedabad, India, 380051\nSection 1 Random paragraph of text goes here. Trying to mimic a collateral document and its spatial and paragraph structure. Just adding one more line to make this a 3-line paragraph and then look at\nthe structure we get.\nSite Improvements\nPrepared By Mohit Shah Ahmedabad,\nAuthorized Person Virat Kohli Delhi, India,\nIndia, 380051\n90123\n15\nInspection Date: 20th January, 2018\nDate of Report: 10th December, 2017\nProperty Name: ABC Elements\nAddress 123 E Random Rd Tempe, Arizona, USA, 85287\n● Size 2500 SF\n● Topography Level\n● Flood Map X500\nIntended User The intended user of this agreement is Mohit Shah. It is to be used to appraising property rights of the client.']


In [18]:
for filename in files:
    file_path = os.path.join(tmp_dir, filename)
    if os.path.exists(file_path):
        with open(file_path, "rb") as fi:
            d = pickle.load(fi)
            texts = d["texts"]
            blocks = d["blocks"]
            kv = d["kv"]
            parser = Reader(params={"src": file_path})
            for item in defs:
                method = item["method"]
                name = item["name"]
                if method == "extract":
                    terms = item["search"]["terms"]
                    include = item["search"]["include"]
                    exclude = item["search"]["exclude"]
                    question = item["search"]["question"]
                    entities = item["types"]["entities"]
                    patterns = item["types"]["patterns"]        
                    c_patterns = [re.compile(p) for p in patterns]
                    pages_list = []
                    if "pages" in item["pages"] and item["pages"] is not None:
                        tmp = item["pages"].split("-")
                        if len(tmp) == 1:
                            pages_list = [int(tmp[0])]
                        elif len(tmp) == 2:
                            pages_list = range(int(tmp[0]), int(tmp[1]))
                    match_found = False
                    answer = None
                    for term in terms:
                        pages, kv_matches, block_matches = parser.search(term, blocks, kv, texts, include=include, exclude=exclude, pages_list=pages_list)
                        for p in pages:
                            page_kv = [tmp for tmp in kv_matches if tmp[0] == p]
                            page_block = [tmp for tmp in block_matches if tmp[0] == p]
                            for kv_item in page_kv:
                                pidx, bidx, data = kv_item
                                for d in data:
                                    key, value = d
                                    for pat in c_patterns:
                                        res = re.search(pat, value)
                                        if res is not None:
                                            match_found = True
                                            answer = value
                                            break
                                    if match_found:
                                        break
                                    if len(entities) == 0:
                                        match_found = True
                                        answer = value
                                        break
                                    doc = nlp(value)
                                    for ent in doc.ents:
                                        if ent.label_ in entities:
                                            answer = ent.text
                                            match_found = True
                                            break                            
                                    if match_found:
                                        break
                                if match_found:
                                    break
                            if match_found:
                                break
                            for block_item in page_block:
                                pidx, bidx, data = block_item
                                for d in data:
                                    value = qa_extractor.extract(question, d)
                                    for pat in c_patterns:
                                        res = re.search(pat, value)
                                        if res is not None:
                                            match_found = True
                                            answer = value
                                            break
                                    if match_found:
                                        break
                                    if len(entities) == 0:
                                        match_found = True
                                        answer = value
                                        break
                                    doc = nlp(value)
                                    for ent in doc.ents:
                                        if ent.label_ in entities:
                                            answer = ent.text
                                            match_found = True
                                            break                            
                                    if match_found:
                                        break                        
                            if match_found:
                                break
                        if match_found:
                            break
                    print (name, answer)
                elif method == "lookup":
                    pass


Inspection Date 20th January, 2018
Report Date December, 2017
Topography Level
